In [1]:
import ibm_db

## Conectar a la credencial del servidor de IBM

In [2]:
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net"
dsn_port = "50000"
dsn_protocol = "TCPIP"
dsn_uid = "jmp64148"
dsn_pwd = "2gqpsmp47j703-pw"

In [3]:
dsn = (
        "DRIVER = {0};"
        "DATABASE = {1};"
        "HOSTNAME = {2};"
        "PORT = {3};"
        "PROTOCOL = {4};"
        "UID = {5};"
        "PWD = {6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

In [4]:
try: 
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database:", dsn_database, "as user:", dsn_uid, "on host:", dsn_hostname)
except: 
    print("Unnable to connect:", ibm_db.conn_errormsg())

Connected to database: BLUDB as user: jmp64148 on host: dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net


## Crear una tabla en la base de datos
    

In [5]:
dropQuery = "drop table INSTRUCTOR"
#Ahora se ejecuta la sentencia drop
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [6]:
#Construir la sentencia Create Table DDL
createQuery ="create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"
createStmt = ibm_db.exec_immediate(conn, createQuery)

# Introducir datos en la tabla

In [7]:
insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"
insertStmt = ibm_db.exec_immediate(conn, insertQuery)


In [8]:
# Insertar dos filas mas en la tabla INSTRUCTOR
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

# Consultar datos desde la tabla

In [9]:
selectQuery = "select * from INSTRUCTOR"
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

In [10]:
#Buscar en el diccionario (solo para la primera fila)
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [11]:
#Buscar en el resto de las filas e imprimir el ID y FNAME de esas filas
while ibm_db.fetch_row(selectStmt) != False:
    print("ID:",  ibm_db.result(selectStmt, 0), "FNAME:",  ibm_db.result(selectStmt, "FNAME"))

ID: 2 FNAME: Raul
ID: 3 FNAME: Hima


In [12]:
#Bonus: escribe ahora y ejecuta una declaración que cambien el campo CITY de los Rav’s por MOOSETOWN

In [13]:
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

# Devolver datos en pandas

In [14]:
import pandas as pd
import ibm_db_dbi

In [15]:
#conexión con pandas
pconn = ibm_db_dbi.Connection(conn)

In [16]:
#Declaración para devolver todas las filas de la tabla INSTRUCTOR\\n",
selectQuery = "select * from INSTRUCTOR"

In [18]:
#devolver los resultados de la busqueda dentro de un dataframe pandas
pdf = pd.read_sql(selectQuery, pconn)

In [19]:
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [21]:
pdf.LNAME[0]

'Ahuja'

In [22]:
pdf.shape

(3, 5)

# Cerrar la conexión con la base de datos

In [23]:
ibm_db.close(conn)

True